# 01 - Quadratic Programs

#### Uncomment the cell below to pip install the necessary modules if not already installed

#### Note: Works with Qiskit Version 1.4.1

In [1]:
# %pip install qiskit==1.4.1
# %pip install qiskit-optimization

#### Restart the kernel after installing any of the missing packages

### Introduction

In this notebook, we will introduce how to build optimization problems using the Qiskit optimization module. Qiskit optimization introduces the `QuadraticProgram` class to make a model of an optimization problem. Optimization problems are seen throughout all scientific fields and typically involve finding the minimum or maximum of a function given initial data and variational parameters that are then optimized to find the best solution. Here, the `QuadraticProgram` deals with quadratically constrained problems given as follows:

$$
\begin{align}
\text{minimize}\quad& x^\top Q_0 x + c^\top x\\
\text{subject to}\quad& A x \leq b\\
& x^\top Q_i x + a_i^\top x \leq r_i, \quad 1,\dots,i,\dots,q\\
& l_i \leq x_i \leq u_i, \quad 1,\dots,i,\dots,n,
\end{align}
$$

- $Q_i$: n x n matrices
- $A$: m x n matrix
- $x$, $c$: n-dimensional vectors
- $b$: m-dimensional vector
- $x$: binary, integer, or continuous variables
- The $\top$ stands for the transpose of the vector/matrix


`QuadracticProgram` supports $\leq$, $\geq$, and $=$ comparisons.

We will primarily focus on introducing the setup for a `QuadraticProgram` and the various functionalities included within the class for carrying out optimization problems.

In [2]:
# This code is from:
# https://qiskit-community.github.io/qiskit-optimization/tutorials/01_quadratic_program.html

### Loading a `QuadraticProgram` from a Linear Programming (LP) file

We'll start by importing the following modules

In [3]:
from qiskit_optimization import QuadraticProgram
from qiskit_optimization.translators import from_docplex_mp

We can then create an empty model. From there we will add variables and constraints to a model. The Qiskit optimization module supports the conversion from Docplex models, so you can easily make a model of an optimization problem with Docplex. For more info on Docplex feel free to check out their documentation [here](https://ibmdecisionoptimization.github.io/docplex-doc/mp/index.html).

We can load a Docplex model to `QuadraticProgram` by using the `from_docplex_mp` function.

In [4]:
# Make a Docplex model
from docplex.mp.model import Model

mdl = Model("docplex model")
x = mdl.binary_var("x")
y = mdl.integer_var(lb=-1, ub=5, name="y")
mdl.minimize(x + 2 * y)
mdl.add_constraint(x - y == 3)
mdl.add_constraint((x + y) * (x - y) <= 1)
print(mdl.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex model

Minimize
 obj: x + 2 y
Subject To
 c1: x - y = 3
 qc1: [ x^2 - y^2 ] <= 1

Bounds
 0 <= x <= 1
 -1 <= y <= 5

Binaries
 x

Generals
 y
End



For readability, we can use `QuadraticProgram`s built in method `prettyprint` to generate a cleaner string representation of the model.

In [5]:
# load from a Docplex model
mod = from_docplex_mp(mdl)
print(type(mod))
print()
print(mod.prettyprint())

<class 'qiskit_optimization.problems.quadratic_program.QuadraticProgram'>

Problem name: docplex model

Minimize
  x + 2*y

Subject to
  Linear constraints (1)
    x - y == 3  'c0'

  Quadratic constraints (1)
    x^2 - y^2 <= 1  'q0'

  Integer variables (1)
    -1 <= y <= 5

  Binary variables (1)
    x



#### Directly constructing a `QuadraticProgram`

We'll now go through how to create an empty model and build it up based on some chosen variables and constraints.

In [6]:
# make an empty problem

mod = QuadraticProgram("my problem")
print(mod.prettyprint())

Problem name: my problem

Minimize
  0

Subject to
  No constraints

  No variables



The `QuadraticProgram` class supports three types of variables

- Binary Variable
- Integer Variable
- Continuous Variable

You can specify variable names, types, and boundary conditions

In [7]:
# Add variables. The lowerbound and upperbound represent the boundaries for the respective variables
# The different variable types are expressed in the function call for the QuadraticProgram instance 

mod.binary_var(name="x")
mod.integer_var(name="y", lowerbound=-1, upperbound=5)
mod.continuous_var(name="z", lowerbound=-1, upperbound=5)
print(mod.prettyprint())

Problem name: my problem

Minimize
  0

Subject to
  No constraints

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



You can set the objective function for the instance of the `QuadraticProgram`. By calling `QuadraticProgram.minimize` or `QuadraticProgram.maximize` you can pass in the objective function you wish to either minimize or maximize. You can add constants, linear, or quadratic objective functions by specifying each type using the keyword arguments for the function. These can be in the form of a list, matrix, or dictionary. 

> **Note**: In the LP format the quadratic part has to be scaled by a factor $1/2$. Thus, when printing as LP format, the quadratic part is first multiplied by 2 and then divided by 2 again. This is done in the background and does not effect the end result.

There are three pieces that have to be specified for quadratic programs. A constant (offset), a linear term ($c^\top x$), and a quadratic term ($x^\top Qx$).

The cell below shows this declaration using a dictionary. For the constant, we can just set it to the integer value we desire. This must be entered as a single value, it will not accept a list of values. For the linear term, the keys in the dictionary correspond to the variable names with the values as the coefficients. For the quadratic term, the keys correspond to the two variables being multiplied (passed in as a tuple), and the values are the coefficients. 

> **Reminder**: Being a quadratic means that the highest order we may have is a power of 2 for any single variable, and no more than two variables can be multiplied together. So we can not have $z \times x \times y$ for example.

In [8]:
# Add objective function using dictionaries

mod.minimize(constant=3, linear={"x": 1}, quadratic={("x", "y"): 2, ("z", "z"): -1})
print(mod.prettyprint())

Problem name: my problem

Minimize
  2*x*y - z^2 + x + 3

Subject to
  No constraints

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



As mentioned earlier, we can also pass our function in as a list/array. The constant term once again must be a single value. The linear term will take in an array that corresponds to the $c$ vector in the mathematical formulation above. The quadratic term takes an array that corresponds to the matrix $Q$. 

The ordering of the variables ($x$ in the mathematical formulation) is the order in which the variables were originally declared in the `QuadraticProgram` object. So since we declared $x$, then $y$, then $z$ if we pass in a list to the linear argument of [1, 0, 0] this will include $x$ but not $y$ or $z$. This also applies to the quadratic term. Since this quadratic term is 2-Dimensional, we pass in a 2D-Array where the interpreted matrix has the following form.

|   | x | y | z |
|:-:|:-:|:-:|:-:|
| x | 0 | 1 | 0 |
| y | 1 | 0 | 0 |
| z | 0 | 0 | -1 |

If we match the rows and columns, this will give us an equation of $xy + yx-z^2 = 2xy-z^2$. With this in mind, we can create our objective function. Feel free to play with this objective function to make sure you understand how it is interpreting the given values of the lists provided.

In [9]:
# Add objective function using lists/arrays
mod.minimize(constant=3, linear=[1, 0, 0], quadratic=[[0, 1, 0], [1, 0, 0], [0, 0, -1]])
print(mod.prettyprint())

Problem name: my problem

Minimize
  2*x*y - z^2 + x + 3

Subject to
  No constraints

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



You can access the constant, linear, and quadratic terms by accessing the `Quadratic.objective.{constant, linear, quadratic}`, respectively. For the linear and quadratic terms, you can get a dense matrix using `to_array`, a sparse matrix using `coefficients`, and a dictionary with `to_dict`. For dictionaries, you can specify whether to use variable indices or names as keys. 

> **Note**: The quadratic terms are stored in a compressed way for a dictionary, e.g., `{('x', 'y'): 1, ('y', 'x'): 2}` is stored as `{('x', 'y'): 3}`.

You can get the quadratic term as a symmetric matrix by calling `to_array(symmetric=True)` or `to_dict(symmetric=True)`. If you call `to_dict(name=True)`, you can get a dictionary whose keys are pairs of variable names.

These various calls are shown in the cell below.

In [10]:
print("constant:\t\t\t", mod.objective.constant)
print("linear dict:\t\t\t", mod.objective.linear.to_dict())
print("linear array:\t\t\t", mod.objective.linear.to_array())
print("linear array as sparse matrix:\n", mod.objective.linear.coefficients, "\n")
print("quadratic dict w/ index:\t", mod.objective.quadratic.to_dict())
print("quadratic dict w/ name:\t\t", mod.objective.quadratic.to_dict(use_name=True))
print(
    "symmetric quadratic dict w/ name:\t",
    mod.objective.quadratic.to_dict(use_name=True, symmetric=True),
)
print("quadratic matrix:\n", mod.objective.quadratic.to_array(), "\n")
print("symmetric quadratic matrix:\n", mod.objective.quadratic.to_array(symmetric=True), "\n")
print("quadratic matrix as sparse matrix:\n", mod.objective.quadratic.coefficients)

constant:			 3
linear dict:			 {np.int32(0): np.int64(1)}
linear array:			 [1 0 0]
linear array as sparse matrix:
 <Dictionary Of Keys sparse matrix of dtype 'int64'
	with 1 stored elements and shape (1, 3)>
  Coords	Values
  (0, 0)	1 

quadratic dict w/ index:	 {(0, 1): np.int64(2), (2, 2): np.int64(-1)}
quadratic dict w/ name:		 {('x', 'y'): np.int64(2), ('z', 'z'): np.int64(-1)}
symmetric quadratic dict w/ name:	 {('x', 'y'): np.int64(1), ('y', 'x'): np.int64(1), ('z', 'z'): np.int64(-1)}
quadratic matrix:
 [[ 0  2  0]
 [ 0  0  0]
 [ 0  0 -1]] 

symmetric quadratic matrix:
 [[ 0  1  0]
 [ 1  0  0]
 [ 0  0 -1]] 

quadratic matrix as sparse matrix:
 <Dictionary Of Keys sparse matrix of dtype 'int64'
	with 2 stored elements and shape (3, 3)>
  Coords	Values
  (0, 1)	2
  (2, 2)	-1


### Adding/Removing Linear and Quadratic Constraints

We have set boundary conditions previously. Now we want to add some constraints. We will go through linear constraints first. These can be set by accessing the `linear_constraint` method of our `QuadraticProgram` instance. The keyword arguments here are `linear`, `sense`, `rhs` and `name`. The `linear` accepts similar parameters as before in either the form of a dictionary or list/array. `sense` is our comparator ($\leq$, $\geq$, or $=$) which are passed in as `<=`, `>=`, and `==`. It will also take `<` or `>`. `rhs` sets the value for the right-hand side of our comparison. `name` is the name of our constraint for reference.

In [11]:
# Add linear constraints
mod.linear_constraint(linear={"x": 1, "y": 2}, sense="==", rhs=3, name="lin_eq")
mod.linear_constraint(linear={"x": 1, "y": 2}, sense="<=", rhs=3, name="lin_leq")
mod.linear_constraint(linear={"x": 1, "y": 2}, sense=">=", rhs=3, name="lin_geq")
print(mod.prettyprint())

Problem name: my problem

Minimize
  2*x*y - z^2 + x + 3

Subject to
  Linear constraints (3)
    x + 2*y == 3  'lin_eq'
    x + 2*y <= 3  'lin_leq'
    x + 2*y >= 3  'lin_geq'

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



We can add quadratic constraints in a similar manner. `quadratic_constraint` can take in linear and quadratic constraints using `linear` and `quadratic` as the keywords. `sense`, `rhs`, and `name` are the same as for `linear_constraint`. For the quadratic constraints, these are passed in the same way they were when we created the objective function above.

In [12]:
# Add quadratic constraints
mod.quadratic_constraint(
    linear={"x": 1, "y": 1},
    quadratic={("x", "x"): 1, ("y", "z"): -1},
    sense="==",
    rhs=1,
    name="quad_eq",
)
mod.quadratic_constraint(
    linear={"x": 1, "y": 1},
    quadratic={("x", "x"): 1, ("y", "z"): -1},
    sense="<=",
    rhs=1,
    name="quad_leq",
)
mod.quadratic_constraint(
    linear={"x": 1, "y": 1},
    quadratic={("x", "x"): 1, ("y", "z"): -1},
    sense=">=",
    rhs=1,
    name="quad_geq",
)
print(mod.prettyprint())

Problem name: my problem

Minimize
  2*x*y - z^2 + x + 3

Subject to
  Linear constraints (3)
    x + 2*y == 3  'lin_eq'
    x + 2*y <= 3  'lin_leq'
    x + 2*y >= 3  'lin_geq'

  Quadratic constraints (3)
    x^2 - y*z + x + y == 1  'quad_eq'
    x^2 - y*z + x + y <= 1  'quad_leq'
    x^2 - y*z + x + y >= 1  'quad_geq'

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



You can access the linear and quadratic terms of the respective constraints in the same way as the objective functions.

In [13]:
lin_geq = mod.get_linear_constraint("lin_geq")
print("lin_geq:", lin_geq.linear.to_dict(use_name=True), lin_geq.sense, lin_geq.rhs)
quad_geq = mod.get_quadratic_constraint("quad_geq")
print(
    "quad_geq:",
    quad_geq.linear.to_dict(use_name=True),
    quad_geq.quadratic.to_dict(use_name=True),
    quad_geq.sense,
    lin_geq.rhs,
)

lin_geq: {'x': np.float64(1.0), 'y': np.float64(2.0)} ConstraintSense.GE 3
quad_geq: {'x': np.float64(1.0), 'y': np.float64(1.0)} {('x', 'x'): np.float64(1.0), ('y', 'z'): np.float64(-1.0)} ConstraintSense.GE 3


You can remove the constraints by making specific calls of `remove_linear_constraint` or `remove_quadratic_constraint` and passing in the `name` of the constraint you want to remove.

In [14]:
# Remove constraints
mod.remove_linear_constraint("lin_eq")
mod.remove_quadratic_constraint("quad_leq")
print(mod.prettyprint())

Problem name: my problem

Minimize
  2*x*y - z^2 + x + 3

Subject to
  Linear constraints (2)
    x + 2*y <= 3  'lin_leq'
    x + 2*y >= 3  'lin_geq'

  Quadratic constraints (2)
    x^2 - y*z + x + y == 1  'quad_eq'
    x^2 - y*z + x + y >= 1  'quad_geq'

  Integer variables (1)
    -1 <= y <= 5

  Continuous variables (1)
    -1 <= z <= 5

  Binary variables (1)
    x



You can substitute some variables with constants or other variables. `QuadraticProgram` has a method `substitute_variables(constants=..., variables=...)` to deal with the following two cases.

- $x \leftarrow c$: when `constants` have a dictionary `{x: c}`
- $x \leftarrow c$: when `variables` have a dictionary `{x: (y, c)}`

In the cell below, we will substitute a constant for the variable $x$ and change the variable for $y$

In [15]:
sub = mod.substitute_variables(constants={"x": 0}, variables={"y": ("z", -1)})
print(sub.prettyprint())

Problem name: my problem

Minimize
  -z^2 + 3

Subject to
  Linear constraints (2)
    -2*z <= 3  'lin_leq'
    -2*z >= 3  'lin_geq'

  Quadratic constraints (2)
    z^2 - z == 1  'quad_eq'
    z^2 - z >= 1  'quad_geq'

  Continuous variables (1)
    -1 <= z <= 1



If the substitution is not possible due to the lower or upper bounds of the variables then the method will return the status `Status.INFEASIBLE`. Here we will try to replace `x` with $-1$, but $-1$ is out of range of `x` as we previously set the range to be between $0$ and $1$.

In [16]:
sub = mod.substitute_variables(constants={"x": -1})
print(sub.status)

Infeasible substitution for variable: x


QuadraticProgramStatus.INFEASIBLE


You can also not substitute variables multiple times. The method raises an error in such a case. Here we will try to substitute `x` twice.

In [17]:
from qiskit_optimization import QiskitOptimizationError

try:
    sub = mod.substitute_variables(constants={"x": -1}, variables={"y": ("x", 1)})
except QiskitOptimizationError as e:
    print("Error: {}".format(e))

Error: 'Cannot substitute by variable that gets substituted itself: y <- x 1'


> **Note**: When you display your problem in LP format using `export_as_lp_string`, `Binaries` denotes binary variables and `Generals` denotes integer variables. If variables are not included in either `Binaries` or `Generals`, such variables are continuous ones with default lower bound = 0 and upper bound = infinity. Also, note that you cannot use 'e' or 'E' as the first character of names due to the [specification of LP format](https://www.ibm.com/docs/en/icos/22.1.0?topic=representation-variable-names-in-lp-file-format).

In [18]:
mod = QuadraticProgram()
mod.binary_var(name="e")
mod.binary_var(name="f")
mod.continuous_var(name="g")
mod.minimize(linear=[1, 2, 3])
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: _e + 2 f + 3 g
Subject To

Bounds
 0 <= _e <= 1
 0 <= f <= 1

Binaries
 _e f
End



### Conclusion

Congratulations! You have created an introductory quadratic equation that can be used for optimization. Building an equation helps to understand the bounds and constraints for the equation/system we are trying to optimize. While we did not explore the actual optimization of the equation, we will do this in many future notebooks! Feel free to play with this notebook and build more equations to help reinforce your understanding of how a `QuadraticProgram` instance functions!.